In [1]:
from dotenv import load_dotenv
import os
import pandas as pd
from pyoso import Client

load_dotenv()
OSO_API_KEY = os.environ['OSO_API_KEY']
client = Client(api_key=OSO_API_KEY)
stringify = lambda arr: "'" + "','".join(arr) + "'"

In [2]:
df = client.to_pandas("""
SELECT
  e.time,
  e.event_type,
  e.event_source,
  e.transaction_hash,
  e.gas_fee,
  e.from_artifact_id
FROM int_superchain_events_by_project AS e
JOIN projects_v1 AS p ON e.project_id = p.project_id
WHERE
  p.display_name = 'Splits'
  AND e.time BETWEEN DATE '2025-03-01' AND DATE '2025-04-01'
""")

In [3]:
df.groupby(['event_type', 'event_source']).agg({
    'transaction_hash': 'nunique',
    'gas_fee': 'sum',
    'from_artifact_id': 'nunique'
}).rename(columns={'from_artifact_id': 'unique_addresses'})

transaction_hash   gas_fee  \
event_type                     event_source                               
CONTRACT_INTERNAL_INVOCATION   BASE                    156193  0.010298   
                               OPTIMISM                  2599  0.000168   
                               ZORA                       506  0.000031   
CONTRACT_INVOCATION            BASE                      7233  0.009446   
                               OPTIMISM                    90  0.000036   
                               ZORA                       152  0.000058   
CONTRACT_INVOCATION_VIA_USEROP BASE                       128  0.000317   
                               OPTIMISM                    93  0.000184   
                               ZORA                        12   0.00002   

                                             unique_addresses  
event_type                     event_source                    
CONTRACT_INTERNAL_INVOCATION   BASE                     12985  
                               OPTIMISM                   868  
                               ZORA                       296  
CONTRACT_INVOCATION            BASE                       194  
                               OPTIMISM                    47  
                               ZORA                       151  
CONTRACT_INVOCATION_VIA_USEROP BASE                        55  
                               OPTIMISM                    45  
                               ZORA                         8